# <font color = 'red'> Analisando Sentimentos das reclamações</font>

In [1]:
!pip install -q textblob

In [6]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from textblob import TextBlob

import warnings
warnings.filterwarnings('ignore')

In [3]:
dados_sentimento = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',')

In [9]:
dados_sentimento.sample(5)

,Empresa,Relato
978,Banco Itaú Unibanco,o itau unibanco incluiu meu cpf na lista do re...
1018,Banco Bradesco,caros boa tarde abri essa conta on line e març...
502,C6 Bank,dia 2592023 recebi uma ligação do banco c6 no ...
642,Banco Inter (Banco Intermedium),mais uma vez estou tentando exercer o meu dire...
1076,Mercado Pago,recebi uma propaganda de promoção de 10 reais ...


In [7]:
### Função para obter subjetividade
def subjetividade(text):
 return TextBlob(text).sentiment.subjectivity

In [8]:
# Função para obter polaridade
def polaridade(text):
 return TextBlob(text).sentiment.polarity

In [10]:
# Adicionando no DF
dados_sentimento['Subjetividade'] = dados_sentimento['Relato'].apply(subjetividade)
dados_sentimento['Polaridade'] = dados_sentimento['Relato'].apply(polaridade)